In [1]:
import numpy as np
import sys, os

In [2]:
#first of all, compile the code where the only change has been this (and the classes are also named differently to make sure there are no clashes when importing both of them together)
path_to_eigen="/Users/rosamartinezcorral/Documents/eigenlibrary/eigen-eigen-323c052e1731/" 
path_to_FPTrepo="\"../\""
file="Ladder_3_v2.cpp"
folder="../bin"
path_1=os.path.join("../", "src")
fname=os.path.join(path_1, file)
objectnamemac=os.path.join(folder,file.replace('.cpp','')) 

compilestringmac="c++ -O2 -DNDEBUG -Wall -shared -std=c++11  -fPIC -undefined dynamic_lookup -I %s -I %s -lmpfr -lmpc `python3 -m pybind11 --includes` %s -o %s`python3-config --extension-suffix`"%(path_to_FPTrepo,path_to_eigen, fname,objectnamemac)

compilestring=compilestringmac
print(compilestring)

!$compilestring

c++ -O2 -DNDEBUG -Wall -shared -std=c++11  -fPIC -undefined dynamic_lookup -I "../" -I /Users/rosamartinezcorral/Documents/eigenlibrary/eigen-eigen-323c052e1731/ -lmpfr -lmpc `python3 -m pybind11 --includes` ../src/Ladder_3_v2.cpp -o ../bin/Ladder_3_v2`python3-config --extension-suffix`
In file included from ../src/Ladder_3_v2.cpp:11:
In file included from /usr/local/include/boost/multiprecision/mpfr.hpp:9:
/usr/local/include/boost/multiprecision/detail/standalone_config.hpp:36:7: warning: "The minimum language standard to use Boost.Math will be C++14 starting in July 2023 (Boost 1.82 release)" [-W#warnings]
 #    warning "The minimum language standard to use Boost.Math will be C++14 starting in July 2023 (Boost 1.82 release)"
      ^
In file included from ../src/Ladder_3_v2.cpp:11:
In file included from /usr/local/include/boost/multiprecision/mpfr.hpp:10:
In file included from /usr/local/include/boost/multiprecision/number.hpp:12:
In file included from /usr/local/include/boost/multipr

In [5]:
sys.path.insert(0, "../bin")
from Ladder_3_i0 import Ladder_3_i0 as L3i0
from Ladder_3_v2 import Ladder_3_v2 as L3v2 #here some small things have been change to speed up: e.g. transpose laplacian once and keep in variable, then use this subsequently

In [6]:
obji0 = L3i0()
objv2 = L3v2()

In [7]:
a12=0.1
b12=0.01
a23=0.1
b23=0.01
a12T=a12*10
a23T=a23*10
b12T=b12
b23T=b23
kon=1
koff=1
P=10
print(koff+b23T)
tf_conc =3
pars=np.array([a12,a23,b12,b23,a12T,a23T,b12T,b23T,kon,koff,P, P])

1.01


In [8]:
#original code
obj=obji0 
obj.setLaplacian_ladder_3(pars) 
obj.mult_laplacian_TF(3)
print(obj.get_FPT_stat(True))
%timeit obj.get_FPT_stat(True)

[[2.97942733]
 [0.65704284]]
755 µs ± 35.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [9]:
#code where the transpose is done once and then used subsequently, plus some other changes like the possibility to not calculate cv of the FPT distribution
obj=objv2
obj.setLaplacian_ladder_3(pars) # super().setLaplacian_ladder_3(converted_pset)-> set_ladder_3_laplacian-> fills laplacian attribute
obj.mult_laplacian_TF(3)
print(obj.get_FPT_stat(True,True))
%timeit obj.get_FPT_stat(True,True)

[[2.97942733]
 [0.65704284]]
682 µs ± 18.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
682/755.

0.9033112582781457

In [11]:
(682+19)/(755-35)

0.9736111111111111

Note that it takes 3-10% less time just with these small changes.

In [12]:
#now do not to compute cv
obj=objv2
obj.setLaplacian_ladder_3(pars) # super().setLaplacian_ladder_3(converted_pset)-> set_ladder_3_laplacian-> fills laplacian attribute
obj.mult_laplacian_TF(3)
print(obj.get_FPT_stat(True,False))
%timeit obj.get_FPT_stat(True,False)

[[ 2.97942733]
 [-1.        ]]
330 µs ± 8.63 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Note that the calculation of the cv takes up half the time. So, if it is not needed, it should not be calculated.